In [1]:
import rasterio
from skimage.exposure import match_histograms

In [2]:
sar_path = "D:/Personal_Projects/Geospatial/Building_Detection/data/spacenet_data/SN6_buildings_AOI_11_Rotterdam_test_public/test_public/AOI_11_Rotterdam/SAR-Intensity/SN6_Test_Public_AOI_11_Rotterdam_SAR-Intensity_20190804111224_20190804111453_tile_8697.tif"
dop_path = "Z:/BuildingDetection/dop_Resample/dop10rgbi_32_280_5652_1_nw_2023.tiff"

In [3]:
# Load SpaceNet 6 SAR-Intensity image (test data)
with rasterio.open(sar_path) as src:
    spacenet_ref = src.read()  # Shape: (bands, height, width)
    spacenet_meta = src.meta

# Load DOP image
with rasterio.open(dop_path) as src:
    dop_image = src.read()
    dop_meta = src.meta

In [ ]:
#  Match Histograms - lign the DOP’s RGB bands with the reference: